In [ ]:
!pip install yargy
!pip install natasha
!pip install pymorphy2==0.8
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [2]:
import yargy
import natasha
import pandas as pd
import numpy as np
import functools
import re
from yargy import Parser
from natasha import AddrExtractor
from natasha import MorphVocab
from natasha.syntax import print_markup
from IPython.display import display
from yargy import (
    rule,
    or_, and_
)
from yargy.interpretation import fact
from yargy.predicates import (
    eq, lte, gte, gram, type, tag,
    length_eq,
    in_, in_caseless, dictionary,
    normalized, caseless,
    is_title
)
from yargy.pipelines import morph_pipeline
from yargy.tokenizer import QUOTES

pd.options.display.max_colwidth = 100

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

###Константы

In [3]:
COUNTRY_KEY_WORDS = ("страна", "россия", "рф")

REGIONS_KEY_WORDS = ("республика", "респ", "рес", "округ", "область", "обл", "об",\
                     "край", "область", "ао", "автономный", "автономная")
REGIONS_KEY_WORDS_BEFORE = ("республика", "респ", "рес", "округ", "область", "обл", "об")
REGIONS_KEY_WORDS_AFTER = ("край", "область", "обл", "об", "ао", "автономный")
SPECIAL_REGIONS = ("краснодарский", "ростовская", "алтайский", "владимирская", \
                   "московская", "нижегородская", "приморский", "калининградская", \
                   "калужская", "кировская", "вологодская", "пензенская", "амурская", \
                   "ставропольский", "ленинградская", "воронежская", "орловская", \
                   "новгородская", "смоленская", "псковская", "ивановская", "волгоградская", \
                   "ярославская", "курганская", "липецкая", "курская", "самарская", "тамбовская", \
                   "тюменская", "омская", "рязанская", "магаданская", "тульская")

TOWNS_KEY_WORDS = ("г", "гор", "город")
SPECIAL_TOWNS = ('октябрьский',)

SETTLEMENT_KEY_WORDS = ("село", "с", "сл", "деревня", "дер", "поселок", "посёлок", "п", "пос", "рп", "пгт", "ст")
SETTLEMENTS_TO_REMOVE = ("пушкина", "ленина", "новгород", "революции", "люксембург", \
                         "энтузиастов", "ермакова", "мира", "чапаев", "ряд", "улица", \
                         "кирова", "гагарина", "энгельса", "калинина", "саха", "фрунзе")
SETTLEMENTS_TO_ADD = ("Зеленоград", "Винсады", "Вольск-18")
COMPLEX_SETTLEMENTS_TO_REMOVE = ("большая дмитровка", "1 мая", "8 марта", "карла маркса",\
                    "красный путь", "розы люксембург", "степана разина", "красный октябрь", \
                    "льва толстого", "красная площадь", "красные зори", "большая покровская", \
                    "максима горького", "красная пресня", "золотая долина")

STREETS_KEY_WORDS = ("улица", "ул", "пр", "проспект", "пр-кт", "пр-т", "проезд", "пр-зд", "пр-д", \
                    "переулок", "пер", "площадь", "пл", "ш", "шоссе", "набережная", "наб", "бульвар", \
                    "б-р", "б", "бул", "бр")

KEY_NAMES_DICT = {"ул": "улица", "пр": "проспект", "пр-кт": "проспект", "пр-зд": "проезд", \
                  "пр-д": "проезд", "пер": "переулок", "пл": "площадь", "наб": "набережная", \
                  "б-р": "бульвар", "бул": "бульвар", "бр": "бульвар"}

# Загрузка данных

Адреса на парсинг

In [4]:
df = pd.read_excel("addresses.xlsx")
df = df[['Column1']]
df = df.dropna()
df = df.drop_duplicates() # потом убрать
df = df.reset_index() # потом убрать
df = df.drop(columns=['index']) # потом убрать
df.rename({"Column1": "address"}, inplace=True, axis=1)

Формируем таблицу из 1000 рандомных записей.

In [ ]:
data = df.copy()

In [ ]:
data = df.sample(n=1000, random_state=21)

Таблица городов России

In [5]:
url = r'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
tables = pd.read_html(url)
table = tables[1]
table.columns = table.columns.droplevel()
wiki_towns = table[['Город', 'Регион']]

Список регионов России

In [6]:
regs = wiki_towns['Регион'].drop_duplicates().to_list()

Таблица сёл, посёлков, деревень

In [7]:
settlements_df = pd.read_excel('locations.xlsx')

# Обработка данных


Удаление знаков препинания, лишних пробелов, слов, замена известных названий, состоящих из 2-ух слов на одну единицу, например, Ясный Бор -> ясный_бор **ПЕРЕДЕЛАТЬ ВСЕ ПОД МНОЖЕСТВА**

In [9]:
def drop_punctuation(s):
    return s.replace(".", " . ").replace(",", " , ")

def drop_spaces(s):
    return re.sub(' +', ' ', s)

def changer(s, dictionary):
    for key, value in dictionary.items():
        s = re.sub(key, value, s, flags=re.IGNORECASE)
    return s

# def chager_dist(s, dictionary):
#     keys = list(dictionary.keys()
#     for word in s.split():
#         if word.lower() in keys:
#             s = re.sub(word, dictionary.get(word.lower()), s, flags=re.IGNORECASE)
#     return s

def locality_dict(locs):
    locality_dict = {}
    for loc in locs:
        if len(re.split(r'[\s]\s*', loc)) > 1:
            name = re.sub(r'\([^()]*\)', "", loc, flags=re.IGNORECASE)
            name = re.split(r'[\s]\s*', name)
            if "" in name:
                name.remove("")
            if " " in name:
                name.remove(" ")
            locality_dict.update({loc: "_".join(name)})
    return locality_dict

def drop_spaces_and_brackets(loc):
    if len(re.split(r'[\s]\s*', loc)) > 1:
        name = re.sub(r'\([^()]*\)', "", loc, flags=re.IGNORECASE)
        name = re.split(r'[\s]\s*', name)
        if "" in name:
            name.remove("")
        if " " in name:
            name.remove(" ")
    else:
        name = (loc,)
    return " ".join(name)

def extract_complex_names(locations):
    result = list()
    extracted = list()
    for loc in locations:
        if len(loc.split()) > 1:
            extracted.append(loc)
        else:
            result.append(loc)
    return result, extracted

def generate_ngrams(s, n):
    s = s.lower()
    s = re.sub(r'^[a-zA-Z0-9,.\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

def locs_changer(s, locs_dict):
    grams_list = generate_ngrams(s, 2)
    for gram in grams_list:
        if locs_dict.get(gram):
            s = re.sub(gram, locs_dict.get(gram), s, flags=re.IGNORECASE)
            break
    return s

def extract_adjf_names(locations):
    result = list()
    extracted = list()
    end_list = ("ий", "ый", "ая", "яя", "ое", "ее", "ой")
    for loc in locations:
        if loc[-2:] in end_list:
            extracted.append(loc)
        else:
            result.append(loc)
    return result, extracted

Обработка списка городов

In [10]:
towns = wiki_towns['Город'].to_list()

change = functools.partial(changer, dictionary={'ё': 'е'})
towns = map(change, towns)
change = functools.partial(changer, dictionary={"оспаривается": ""})
towns = list(map(change, towns))
towns_dict = locality_dict(towns)
change = functools.partial(changer, dictionary=towns_dict)
towns = list(map(change, towns))
towns = list(dict.fromkeys(towns))

Обработка списка регионов

In [11]:
regions = list()
for reg in regs:
    reg = reg.lower()
    reg = reg.replace(" ао", "").replace(" — югра", "").replace(" — алания", "").replace(" область", "").replace(" край", "")
    regions.append("_".join(re.split(r'[\s]\s*', reg)))
regions.append("чувашская")
regions.append("саха")
regions.remove("москва")
regions.remove("санкт-петербург")

Обработка списка поселений

In [12]:
settlements = settlements_df['location'].to_list()
for elem in list(SETTLEMENTS_TO_ADD):
    settlements.append(elem)

change = functools.partial(changer, dictionary={'ё': 'е'})
settlements = list(map(change, settlements))
settlements = list(map(drop_spaces_and_brackets, settlements))
settlements = list(map(str.lower, settlements))
settlements = list(dict.fromkeys(settlements))
settlements, complex_settlements = extract_complex_names(settlements)
settlements, settlements_adjf = extract_adjf_names(settlements)

for elem in list(SETTLEMENTS_TO_REMOVE):
    if elem in settlements:
        settlements.remove(elem)

for elem in list(COMPLEX_SETTLEMENTS_TO_REMOVE):
    if elem in complex_settlements:
        complex_settlements.remove(elem)

complex_settlements_dict = locality_dict(complex_settlements)
complex_list = list(complex_settlements_dict.values())
settlements_list = settlements + complex_list

In [95]:
data = df.sample(n=100, random_state=50)
data['address'] = data['address'].apply(str)
data['address'] = data['address'].apply(drop_punctuation)
data['address'] = data['address'].apply(drop_spaces)
data['address'] = data['address'].apply(changer, dictionary={'ё': 'е'})
data['address'] = data['address'].apply(changer, dictionary=towns_dict)
data['address'] = data['address'].apply(locs_changer, locs_dict=complex_settlements_dict)
# data['address'] = data['address'].apply(changer_dist, dictionary=KEY_NAMES_DICT)#ПРОВЕРИТЬ КАК ЭТО РАБОТАЕТ

Формирование таблицы, которая будет заполняться найденными сущностями

In [96]:
data['region'] = ""
data['town'] = ""
data['district'] = ""
data['settlement'] = ""
data['street'] = ""
data['building'] = ""
data['index'] = ""
data['place'] = ""
data['none'] = ""
data['else'] = ""

# Парсинг

###Функции

In [131]:
#заменить северную осетию
def regions_extractor(s, names, types_before=None, types_after=None, special_names=None):
    #names = list(map(str.lower, names))
    addr_type_before = 0
    addr_type_after = 0
    name = None
    for word in s.lower().split():
        if word in names and not word in special_names:
            return word.capitalize()
        elif special_names and word in special_names:
            if addr_type_before == 1:
                return word.capitalize()
            else:
                name = word
                addr_type_after = 1
                continue
        elif word in types_after and addr_type_after:
            return name.capitalize()
        elif word in types_before:
            addr_type_before = 1
            continue
        if addr_type_after:
            addr_type_after = 0
        if addr_type_before:
            addr_type_before = 0
    return ""

def addr_part_extractor(s, names, types=None, special_names=list()):
    names = list(map(str.lower, names))
    addr_type = False
    for word in s.split():
        if types:
            if word.lower() in types:
                addr_type = True
            elif word.lower() in special_names and addr_type or word.lower() in names and not word.lower() in special_names:
                return word.capitalize()
        else:
            if word.lower() in names:
                return word.capitalize()
    return ""

def delete_addr_part(s, addr_name, addr_type):
    if addr_name == "" or addr_name[0] == "":
        return(s)
    result = list()
    deleted = list()
    addr_name = list(map(str.lower, addr_name))
    addr_type = list(map(str.lower, addr_type))
    key = False
    # удаляем повторяющиеся названия, но стараемся не трогать другие совпадающие названия
    for word in s.split():
        if not word.lower() in addr_name and not word.lower() in addr_type:
            result.append(word)
        elif key and word.lower() in addr_name and not word.lower() in deleted:
            result.append(word)
        elif not key and word.lower() in addr_name:
            key = True
            deleted.append(word.lower())
    return " ".join(result)

def remove_duplicate_punctuation(s):
    res = list()
    for word in s.split():
        if res:
            if res[-1] != ',' and res[-1] != '.' or word.isalnum():
                word = re.sub(',+', ',', word)
                word = re.sub('\.+', '.', word)
                res.append(word)
        elif word.isalnum():
                res.append(word)
    return " ".join(res)

###Страны

In [98]:
data['else'] = data.apply(lambda x: delete_addr_part(x['address'], COUNTRY_KEY_WORDS, COUNTRY_KEY_WORDS), axis=1)

###Города

In [99]:
data['town'] = data['else'].apply(addr_part_extractor, names=towns, types=TOWNS_KEY_WORDS, special_names=SPECIAL_TOWNS)
data['else'] = data.apply(lambda x: delete_addr_part(x['else'], (x['town'],), TOWNS_KEY_WORDS), axis=1)

###Регионы

In [100]:
data['region'] = data['else'].apply(regions_extractor, names=regions, \
                                       types_before=REGIONS_KEY_WORDS_BEFORE, \
                                       types_after=REGIONS_KEY_WORDS_AFTER, \
                                       special_names=SPECIAL_REGIONS)
data['else'] = data.apply(lambda x: delete_addr_part(x['else'], (x['region'],), REGIONS_KEY_WORDS), axis=1)

###Поселения

In [101]:
%%time
data['settlement'] = data[data['town']==""]['else'].apply(addr_part_extractor, names=settlements_list, types=SETTLEMENT_KEY_WORDS)
data = data.fillna(str(""))
data['else'] = data.apply(lambda x: delete_addr_part(x['else'], (x['settlement'],), SETTLEMENT_KEY_WORDS), axis=1)

CPU times: user 119 ms, sys: 14 ms, total: 133 ms
Wall time: 139 ms


#Yargy

###Yargy правила

In [102]:
def value(key):
        @property
        def field(self):
            return getattr(self, key)
        return field

In [103]:
from natasha.grammars.addr import GOROD, INDEX, FED_OKRUG, RESPUBLIKA, KRAI, AUTO_OKRUG

from yargy import Parser
from IPython.display import display
from yargy import (
    rule,
    or_, and_
)
from yargy.interpretation import fact
from yargy.predicates import (
    eq, lte, gte, gram, type, tag,
    length_eq,
    in_, in_caseless, dictionary,
    normalized, caseless,
    is_title
)
from yargy.pipelines import morph_pipeline
from yargy.tokenizer import QUOTES

INT = type('INT')
DOT = eq('.')
ADJF = gram('ADJF')
NOUN = gram('NOUN')
TITLE = is_title()
DASH = eq('-')
SLASH = eq('/')
QUOTE = in_(QUOTES)

ANUM = rule(
    INT,
    DASH.optional(),
    in_caseless({
        'я', 'й', 'е',
        'ое', 'ая', 'ий', 'ой'
    })
)

def value(key):
    @property
    def field(self):
        return getattr(self, key)
    return field

work_place = fact(
    'work_place',
    ['number', 'type']
)

AddrPart = fact(
    'AddrPart',
    ['value']
)

work_place_part = fact(
    'AddrPart',
    ['value']
)

OnlyNumberIsu = fact(
    'OnlyNumberIsu',
    ['number']
)

OnlyNumberPlace = fact(
    'OnlyNumberPlace',
    ['number']
)

OnlyNameBC = fact(
    'OnlyNameBC',
    ['name']
)

Region = fact(
    'Region',
    ['name', 'type']
)

Building = fact(
    'Building',
    ['number', 'type']
)

Room = fact(
    'Room',
    ['number', 'type']
)

Settlement = fact(
    'Settlement',
    ['name', 'type']
)

Street = fact(
    'Street',
    ['name', 'type']
)

class Region(Region):
    value = value('name')

class Settlement(Settlement):
    value = value('name')
    
class Street(Settlement):
    value = value('name')

class Building(Building):
    value = value('number')
    
class Room(Room):
    value = value('number')

class OnlyNumberIsu(OnlyNumberIsu):
    type = 'ВСП/ДО'
    value = value('number')
    
class OnlyNumberPlace(OnlyNumberPlace):
    type = 'место'
    value = value('number')   
    
class OnlyNameBC(OnlyNameBC):
    type = 'БЦ'
    value = value('name')    
    
class work_place(work_place):
    value = value('number')   
    
class work_place_part(AddrPart):
    @property
    def obj(self):
        from natasha import obj

        part = self.value
        return obj.AddrPart(part.value, part.type)

from yargy.relations import gender_relation
from yargy.relations.constructors import Relation

class new_gender_relation(Relation):
    label = 'gender_fem'

    def __call__(self, form, other):
        if form.grams.number.plural and other.grams.number.plural:
            return True
        
        fem_types = ("ул", "пл", "наб")
        male_types = ("пр", "пр-кт", "пр-т", "пр-зд", "пр-д", \
                    "пер", "б-р", "б", "бул", "бр")
        neut_types = ("ш",)
        (form_male, form_female, form_neutral,
         form_bi, form_general) = form.grams.gender
        (other_male, other_female, other_neutral,
         other_bi, other_general) = other.grams.gender
        if form.normalized in fem_types:
            form_female = True
        if other.normalized in fem_types:
            other_female = True
        if form.normalized in male_types:
            form_male = True
        if other.normalized in male_types:
            other_male = True
        if form.normalized in neut_types:
            form_neutral = True
        if other.normalized in neut_types:
            other_neutral = True
        return (
            (form_male and other_male)
            or (form_female and other_female)
            or (form_neutral and other_neutral)
            or (form_bi and (other_male or other_female))
            or (other_bi and (form_male or form_female))
            or form_general
            or other_general)
        

#########
#
#  Number
#
##########

LETTER = in_caseless(set('абвгдежзийлмнопрстуфхшщэюя'))
LETTER_LATIN = in_caseless(set('qwertyuiopasdfghjlzxcvbnm'))
LETTERS = or_(
    rule(LETTER),
    rule(LETTER_LATIN)
)
SEP = in_(r'/\-')

SIMPLE_NUMBER = rule(
    INT,
    LETTERS.optional()
)

NUMBER_CASES = or_(
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS)
)

NUMBER = rule(
    eq('№').optional(),
    NUMBER_CASES
)

#########
#
#  BC
#
##########

SIMPLE_BC = or_(
        NOUN,
        ADJF
)

COMPLEX_BC = morph_pipeline([
    'poklonka place',
    'президент плаза',
    'поклонка плейс',
    'северное сияние',
    'даниловская мануфактура',
    'даниловский форт',
    'корпус кнопа',
    'чайка плаза',
    'романов двор-2',
    'золотая долина',
    'Фаберже 8'
])

COMPLEX_BC_WORDS = morph_pipeline([
    'бизнес-центр',
    'бизнес центр',
    'бизнес-парк'
])

SIMPLE_BC_KNOWN = dictionary({
    'кнопа',
    'ртс',
    'Чайка',
    'сенатор'
})

BC_ABBR = in_caseless({
    'ДМ'
})

ONLY_NAME_BC = or_(
   COMPLEX_BC,
   rule(SIMPLE_BC_KNOWN)
).interpretation(
    OnlyNameBC.name
).interpretation(
    OnlyNameBC
)

BC_NAME = or_(
    rule(SIMPLE_BC_KNOWN),
    rule(BC_ABBR),
    rule(QUOTE, SIMPLE_BC_KNOWN, QUOTE),
    rule(QUOTE, COMPLEX_BC, QUOTE),
    rule(QUOTE, BC_ABBR, QUOTE),
    rule(QUOTE, SIMPLE_BC, QUOTE),
    rule(SIMPLE_BC),
    rule(COMPLEX_BC)
).interpretation(
    Settlement.name
)

BC_WORDS = or_(
        rule(caseless('бц'),
        DOT.optional()),
        COMPLEX_BC_WORDS
).interpretation(
    Settlement.type.const('БЦ')
)

BC = or_(
    rule(BC_WORDS, BC_NAME),
    rule(
        COMPLEX_BC_WORDS,
        BC_NAME
    )
).interpretation(
    Settlement
)

#########
#
#  Floor
#
##########

FLOORNAME = rule(caseless('этаж')
).interpretation(
    work_place.type.const('этаж'))

FLOOR_NUMBER = NUMBER.interpretation(
    work_place.number
)

FLOOR = rule(
    FLOORNAME,
    FLOOR_NUMBER
).interpretation(
    work_place
)

#########
#
#  Cabinet
#
##########


CABNAME = or_(
    rule(caseless('кабинет')),
    rule(
        caseless('каб'),
        DOT.optional()
    ),
    rule(
        caseless('к'),
        DOT
    )
).interpretation(
    work_place.type.const('кабинет')
)

CAB_NUMBER = NUMBER.interpretation(
    work_place.number
)

CABINET = rule(
    CABNAME,
    CAB_NUMBER
).interpretation(
    work_place
)

#########
#
#  Sector
#
##########

DOT_INT = rule(DOT, INT)

BLOCKNAME = or_(
    rule(caseless('блок')),
    rule(caseless('башня'))
).interpretation(
    work_place.type.const('Блок'))

SECTORNAME = or_(
    rule(caseless('сектор')),
    rule(caseless('секция')),
    rule(caseless('помещение')),
    rule(caseless('пом'), DOT.optional()),
).interpretation(
    work_place.type.const('сектор'))

COMPLEX_SECTORNAME = morph_pipeline([
    'доп.помещение',
    'дополнительное помещение',
    'служебное помещение'
])

LETTERS_OPT_INT = or_(
    rule(LETTER, INT.optional()),
    rule(LETTER_LATIN, INT.optional())
)

LETTERS_INT = or_(
    rule(LETTER, INT),
    rule(LETTER_LATIN, INT)
)

LETTERS_INTS = rule(LETTERS_OPT_INT, DOT_INT)

QUOTE_LETTER_INT = rule(
    QUOTE, LETTERS_OPT_INT, QUOTE
)

SECTOR_NUMBER = or_(
    rule(QUOTE_LETTER_INT),
    rule(LETTERS_INT, DOT_INT.optional()),
    rule(LETTERS_OPT_INT),
    rule(LETTERS, DOT_INT),
    rule(INT, DOT_INT, DOT_INT.optional()),
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS)
).interpretation(
    work_place.number
)

SECTOR = or_(
    rule(SECTORNAME,
    eq('№').optional(),SECTOR_NUMBER),
    rule(BLOCKNAME,
    eq('№').optional(),SECTOR_NUMBER),
    rule(COMPLEX_SECTORNAME,
    eq('№').optional(),SECTOR_NUMBER),
).interpretation(
    work_place
)

#########
#
#  Room
#
##########

ROOMNAME = or_(
    rule(caseless('комната')),
    rule(caseless('ком'),
         DOT.optional()),
    rule(caseless('комн'),
         DOT.optional())
).interpretation(
    work_place.type.const('комната'))

ROOM_NUMBER = or_(
    rule(LETTERS_INT),
    rule(INT, DOT_INT, DOT_INT.optional()),
    rule(SIMPLE_NUMBER)
).interpretation(
    work_place.number
)

ROOM = rule(
    ROOMNAME,
    eq('№').optional(),ROOM_NUMBER
).interpretation(
    work_place
)

#########
#
#  Window
#
##########

ROWNAME = rule(caseless('ряд')
).interpretation(
    work_place.type.const('ряд'))

WINDOWNAME = rule(normalized('окно')
).interpretation(
    work_place.type.const('окно'))

WINDOW_NUMBER = NUMBER.interpretation(
    work_place.number
)

WINDOW = or_(
    rule(WINDOWNAME,
    WINDOW_NUMBER),
    rule(ROWNAME, WINDOW_NUMBER)
).interpretation(
    work_place
)

#########
#
#  ВСП
#
##########

INT_FOUR = and_(INT,
                length_eq(4))

ONLY_NUMBER_ISU = rule(
    INT_FOUR,
    SLASH,
    INT
).interpretation(
    OnlyNumberIsu.number
).interpretation(
    OnlyNumberIsu
)

BRANCHNAME = or_(rule(ADJF.optional(),caseless('отделение')),
                 rule(caseless('ОСБ')),
                 rule(caseless('ГОСБ'))
).interpretation(
    work_place.type.const('Отделение'))

COMPLEX_ISUNAME = morph_pipeline([
    'дополнительный офис',
    'доп.офис',
    'Д/О'
])

ISUNAME = or_(
    rule(caseless('ВСП')),
    rule(caseless('ДО')),
    rule(caseless('УДО')),
    rule(caseless('СДО')),
    rule(caseless('ДФ')),
    COMPLEX_ISUNAME
).interpretation(
    work_place.type.const('ВСП/ДО'))

ISU_NUMBER = or_(
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS),
).interpretation(
    work_place.number
)

COMPLEX_ISU = rule(COMPLEX_ISUNAME, 
         eq('№').optional(),
         ISU_NUMBER)

ISU = or_(
    rule(ISUNAME,
         eq('№').optional(),
         ISU_NUMBER),
    rule(BRANCHNAME,
         eq('№').optional(),
         ISU_NUMBER),
    rule(COMPLEX_ISU)
).interpretation(
    work_place
)

############
#
#    OBLAST IMP
#
############


OBLAST_WORDS = or_(
    rule(normalized('область')),
    rule(
        caseless('обл'),
        DOT.optional()
    )
).interpretation(
    Region.type.const('область')
)

OBLAST_NAME = dictionary({
    'амурский',
    'архангельский',
    'астраханский',
    'белгородский',
    'брянский',
    'владимирский',
    'волгоградский',
    'вологодский',
    'воронежский',
    'горьковский',
    'ивановский',
    'ивановский',
    'иркутский',
    'калининградский',
    'калужский',
    'камчатский',
    'кемеровский',
    'кировский',
    'костромской',
    'курганский',
    'курский',
    'ленинградский',
    'липецкий',
    'магаданский',
    'московский',
    'мурманский',
    'нижегородский',
    'новгородский',
    'новосибирский',
    'омский',
    'оренбургский',
    'орловский',
    'пензенский',
    'пермский',
    'псковский',
    'ростовский',
    'рязанский',
    'самарский',
    'саратовский',
    'сахалинский',
    'свердловский',
    'смоленский',
    'тамбовский',
    'тверской',
    'томский',
    'тульский',
    'тюменский',
    'ульяновский',
    'челябинский',
    'читинский',
    'ярославский',
}).interpretation(
    Region.name
)

OBLAST = or_(
    rule(OBLAST_NAME, OBLAST_WORDS),
    rule(OBLAST_WORDS, OBLAST_NAME)
).interpretation(
    Region
)

##########
#
#  RAION IMP
#
###########


RAION_WORDS = or_(
    rule(caseless('р'), '-', in_caseless({'он', 'н'})),
    rule(caseless('мрн')),
    rule(caseless('мкр')),
    rule(normalized('район')),
    rule(normalized('микрорайон'))
).interpretation(
    Region.type.const('район')
)

RAION_SIMPLE_NAME = and_(
    ADJF
)

RAION_MODIFIERS = rule(
    in_caseless({
        'усть',
        'северо',
        'александрово',
        'гаврилово',
    }),
    DASH.optional()
)

RAION_COMPLEX_NAME = rule(
    RAION_MODIFIERS,
    RAION_SIMPLE_NAME
)

RAION_NAME = or_(
    rule(RAION_SIMPLE_NAME),
    RAION_COMPLEX_NAME
).interpretation(
    Region.name
)

RAION = rule(
    RAION_NAME,RAION_WORDS
    #rule(RAION_WORDS,RAION_NAME)
).interpretation(
    Region
)

##########
#
#  SETTLEMENT NAME IMP
#
##########


ADJS = gram('ADJS')
SIMPLE = or_(
        NOUN,  # Александровка, Заречье, Горки
        ADJS,  # Кузнецово
        ADJF,  # Никольское, Новая, Марьино
)

COMPLEX = rule(
    SIMPLE,
    DASH.optional(),
    SIMPLE
)

NAME = or_(
    rule(SIMPLE),
    COMPLEX
)

SETTLEMENT_NAME = or_(
    NAME,
    rule(NAME, '-', INT),
    rule(NAME, ANUM)
)

###########
#
#   SELO CPY
#
#############


SELO_WORDS = or_(
    rule(
        caseless('с'),
        DOT.optional()
    ),
    rule(normalized('село'))
).interpretation(
    Settlement.type.const('село')
)

SELO_NAME = SETTLEMENT_NAME.interpretation(
    Settlement.name
)

SELO = rule(
    SELO_WORDS,
    SELO_NAME
).interpretation(
    Settlement
)

###########
#
#   DEREVNYA  IMP
#
#############


DEREVNYA_WORDS = or_(
    rule(
        caseless('д'),
        DOT.optional()
    ),
    rule(
        caseless('ст'),
        DOT.optional()
    ),
    rule(normalized('деревня')),
    rule(normalized('станица'))
).interpretation(
    Settlement.type.const('деревня')
)

DEREVNYA_NAME = SETTLEMENT_NAME.interpretation(
    Settlement.name
)

DEREVNYA = rule(
    DEREVNYA_WORDS,
    DEREVNYA_NAME
).interpretation(
    Settlement
)

###########
#
#   POSELOK IMP
#
#############


POSELOK_WORDS = or_(
    rule(
        in_caseless({'п', 'пос'}),
        DOT.optional()
    ),
    rule(normalized('посёлок')),
    rule(
        caseless('р'),
        DOT.optional(),
        caseless('п'),
        DOT.optional()
    ),
    rule(
        caseless('рп'),
        DOT.optional()
    ),
    rule(
        normalized('рабочий'),
        normalized('посёлок')
    ),
    rule(
        caseless('пгт'),
        DOT.optional()
    ),
    rule(
        caseless('п'), DOT, caseless('г'), DOT, caseless('т'),
        DOT.optional()
    ),
    rule(
        normalized('посёлок'),
        normalized('городского'),
        normalized('типа'),
    ),
).interpretation(
    Settlement.type.const('посёлок')
)

POSELOK_NAME = SETTLEMENT_NAME.interpretation(
    Settlement.name
)

POSELOK = rule(
    POSELOK_WORDS,
    POSELOK_NAME
).interpretation(
    Settlement
)

##############
#
#   ADDR PERSON CPY
#
############


ABBR = and_(
    length_eq(1),
    is_title()
)

PART = and_(
    TITLE,
    or_(
        gram('Name'),
        gram('Surn')
    )
)

MAYBE_FIO = or_(
    rule(TITLE, PART),
    rule(PART, TITLE),
    rule(ABBR, '.', TITLE),
    rule(ABBR, '.', ABBR, '.', TITLE),
    rule(TITLE, ABBR, '.', ABBR, '.')
)

POSITION_WORDS_ = or_(
    rule(
        dictionary({
            'мичман',
            'геолог',
            'подводник',
            'краевед',
            'снайпер',
            'штурман',
            'бригадир',
            'учитель',
            'политрук',
            'военком',
            'ветеран',
            'историк',
            'пулемётчик',
            'авиаконструктор',
            'адмирал',
            'академик',
            'актер',
            'актриса',
            'архитектор',
            'атаман',
            'врач',
            'воевода',
            'генерал',
            'губернатор',
            'хирург',
            'декабрист',
            'разведчик',
            'граф',
            'десантник',
            'конструктор',
            'скульптор',
            'писатель',
            'поэт',
            'капитан',
            'князь',
            'комиссар',
            'композитор',
            'космонавт',
            'купец',
            'лейтенант',
            'лётчик',
            'майор',
            'маршал',
            'матрос',
            'подполковник',
            'полковник',
            'профессор',
            'сержант',
            'старшина',
            'танкист',
            'художник',
            'герой',
            'княгиня',
            'строитель',
            'дружинник',
            'диктор',
            'прапорщик',
            'артиллерист',
            'графиня',
            'большевик',
            'патриарх',
            'сварщик',
            'офицер',
            'рыбак',
            'брат',
        })
    ),
    rule(normalized('генерал'), normalized('армия')),
    rule(normalized('герой'), normalized('россия')),
    rule(
        normalized('герой'),
        normalized('российский'), normalized('федерация')),
    rule(
        normalized('герой'),
        normalized('советский'), normalized('союз')
    ),
)

ABBR_POSITION_WORDS = rule(
    in_caseless({
        'адм',
        'ак',
        'акад',
    }),
    DOT.optional()
)

POSITION_WORDS = or_(
    POSITION_WORDS_,
    ABBR_POSITION_WORDS
)

MAYBE_PERSON = or_(
    MAYBE_FIO,
    rule(POSITION_WORDS, MAYBE_FIO),
    rule(POSITION_WORDS, TITLE)
)

###########
#
#   IMENI CPY
#
##########


IMENI_WORDS = or_(
    rule(
        caseless('им'),
        DOT.optional()
    ),
    rule(caseless('имени'))
)

IMENI = or_(
    rule(
        IMENI_WORDS.optional(),
        MAYBE_PERSON
    ),
    rule(
        IMENI_WORDS,
        TITLE
    )
)

##########
#
#   LET CPY
#
##########


LET_WORDS = or_(
    rule(caseless('лет')),
    rule(
        DASH.optional(),
        caseless('летия')
    )
)

LET_NAME = in_caseless({
    'влксм',
    'ссср',
    'алтая',
    'башкирии',
    'бурятии',
    'дагестана',
    'калмыкии',
    'колхоза',
    'комсомола',
    'космонавтики',
    'москвы',
    'октября',
    'пионерии',
    'победы',
    'приморья',
    'района',
    'совхоза',
    'совхозу',
    'татарстана',
    'тувы',
    'удмуртии',
    'улуса',
    'хакасии',
    'целины',
    'чувашии',
    'якутии',
})

LET = rule(
    INT,
    LET_WORDS,
    LET_NAME
)

##########
#
#    ADDR DATE CPY
#
#############


MONTH_WORDS = dictionary({
    'январь',
    'февраль',
    'март',
    'апрель',
    'май',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь',
})

DAY = and_(
    INT,
    gte(1),
    lte(31)
)

YEAR = and_(
    INT,
    gte(1),
    lte(2100)
)

YEAR_WORDS = normalized('год')

DATE = or_(
    rule(DAY, MONTH_WORDS),
    rule(YEAR, YEAR_WORDS)
)

#########
#
#   MODIFIER CPY
#
############


MODIFIER_WORDS_ = rule(
    dictionary({
        'большой',
        'малый',
        'средний',

        'верхний',
        'центральный',
        'нижний',
        'северный',
        'дальний',

        'первый',
        'второй',

        'старый',
        'новый',

        'красный',
        'лесной',
        'тихий',
    }),
    DASH.optional()
)

ABBR_MODIFIER_WORDS = rule(
    in_caseless({
        'б', 'м', 'н'
    }),
    DOT.optional()
)

SHORT_MODIFIER_WORDS = rule(
    in_caseless({
        'больше',
        'мало',
        'средне',

        'верх',
        'верхне',
        'центрально',
        'нижне',
        'северо',
        'дальне',
        'восточно',
        'западно',

        'перво',
        'второ',

        'старо',
        'ново',

        'красно',
        'тихо',
        'горно',
    }),
    DASH.optional()
)

MODIFIER_WORDS = or_(
    MODIFIER_WORDS_,
    ABBR_MODIFIER_WORDS,
    SHORT_MODIFIER_WORDS,
)



##########
#
#   ADDR NAME IMP
#
##########


ROD = gram('gent')

SIMPLE_NOUN = and_(NOUN, ROD)

COMPLEX = or_(
    rule(
        and_(ADJF),
        NOUN
    ),
    rule(
        TITLE,
        DASH.optional(),
        TITLE
    ),
)

EXCEPTION = dictionary({
    'арбат',
    'варварка',
    'мельникайте',
    'каховка',
    'зорге'
})

MAYBE_NAME = or_(
    rule(SIMPLE_NOUN),
    COMPLEX,
    rule(EXCEPTION)
)

NAME = or_(
    MAYBE_NAME,
    LET,
    DATE,
    IMENI
)

NAME = rule(
    MODIFIER_WORDS.optional(),
    NAME
)

ADDR_CRF = tag('I').repeatable()

NAME = or_(
    NAME,
    ANUM,
    rule(NAME, ANUM),
    rule(ANUM, NAME),
    rule(INT, DASH.optional(), NAME),
    rule(NAME, DASH, INT),
    ADDR_CRF
)

ADDR_NAME = NAME


########
#
#    STREET IMP
#
#########

gender = new_gender_relation()

STREET_WORDS = or_(
    rule(normalized('улица')),
    rule(
        caseless('ул'),
        DOT.optional()
    )
).interpretation(
    Street.type.const('улица')
).match(gender)


STREET_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

STREET = or_(
    rule(STREET_WORDS, STREET_NAME),
    rule(STREET_NAME, STREET_WORDS)
).interpretation(
    Street
)

##########
#
#    PROSPEKT IMP
#
##########


PROSPEKT_WORDS = or_(
    rule(
        in_caseless({'пр', 'просп'}),
        DOT.optional()
    ),
    rule(
        caseless('пр'),
        '-',
        in_caseless({'кт', 'т'}),
        DOT.optional()
    ),
    rule(normalized('проспект'))
).interpretation(
    Street.type.const('проспект')
).match(gender)

PROSPEKT_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PROSPEKT = or_(
    rule(PROSPEKT_WORDS, PROSPEKT_NAME),
    rule(PROSPEKT_NAME, PROSPEKT_WORDS)
).interpretation(
    Street
)


############
#
#    PROEZD IMP
#
#############


PROEZD_WORDS = or_(
    rule(caseless('пр'), DOT.optional()),
    rule(
        caseless('пр'),
        '-',
        in_caseless({'зд', 'д'}),
        DOT.optional()
    ),
    rule(normalized('проезд'))
).interpretation(
    Street.type.const('проезд')
).match(gender)

PROEZD_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PROEZD = or_(
    rule(PROEZD_WORDS, PROEZD_NAME),
    rule(PROEZD_NAME, PROEZD_WORDS)
).interpretation(
    Street
)


###########
#
#   PEREULOK IMP
#
##############


PEREULOK_WORDS = or_(
    rule(
        caseless('п'),
        DOT
    ),
    rule(
        caseless('пер'),
        DOT.optional()
    ),
    rule(normalized('переулок'))
).interpretation(
    Street.type.const('переулок')
).match(gender)

PEREULOK_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PEREULOK = or_(
    rule(PEREULOK_WORDS, PEREULOK_NAME),
    rule(PEREULOK_NAME, PEREULOK_WORDS)
).interpretation(
    Street
)


########
#
#  PLOSHAD IMP
#
##########


PLOSHAD_WORDS = or_(
    rule(
        caseless('пл'),
        DOT.optional()
    ),
    rule(normalized('площадь'))
).interpretation(
    Street.type.const('площадь')
).match(gender)

PLOSHAD_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PLOSHAD = or_(
    rule(PLOSHAD_WORDS, PLOSHAD_NAME),
    rule(PLOSHAD_NAME, PLOSHAD_WORDS)
).interpretation(
    Street
)

########
#
#  ADDR VALUE IMP
#
##########

LETTER = or_(
    rule(LETTER),
    rule(QUOTE, LETTER, QUOTE)
)

VALUE = rule(
    INT,
    LETTER.optional()
)

SEP = in_(r'/\-')

VALUE = or_(
    rule(VALUE),
    rule(VALUE, SEP, VALUE),
    rule(VALUE, SEP, LETTER),
)

ADDR_VALUE = rule(
    eq('№').optional(),
    VALUE
)

############
#
#   SHOSSE IMP
#
###########

SHOSSE_WORDS = or_(
    rule(
        caseless('ш'),
        DOT.optional()
    ),
    rule(normalized('шоссе'))
).interpretation(
    Street.type.const('шоссе')
).match(gender)

SHOSSE_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

SHOSSE = or_(
    rule(SHOSSE_NAME, SHOSSE_WORDS),
    rule(SHOSSE_WORDS, SHOSSE_NAME)
).interpretation(
    Street
)

########
#
#  NABEREG IMP
#
##########


NABEREG_WORDS = or_(
    rule(
        caseless('наб'),
        DOT.optional()
    ),
    rule(normalized('набережная'))
).interpretation(
    Street.type.const('набережная')
).match(gender)

NABEREG_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

NABEREG = or_(
    rule(NABEREG_WORDS, NABEREG_NAME),
    rule(NABEREG_NAME, NABEREG_WORDS)
).interpretation(
    Street
)


########
#
#  BULVAR IMP
#
##########


BULVAR_WORDS = or_(
    rule(
        caseless('б'),
        '-',
        caseless('р'),
        DOT.optional()
    ),
    rule(
        caseless('б'),
        DOT
    ),
    rule(
        caseless('бул'),
        DOT.optional()
    ),
    rule(normalized('бульвар'))
).interpretation(
    Street.type.const('бульвар')
).match(gender)

BULVAR_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

BULVAR = or_(
    rule(BULVAR_WORDS, BULVAR_NAME),
    rule(BULVAR_NAME, BULVAR_WORDS)
).interpretation(
    Street
)


############
#
#    DOM IMP
#
#############


DOM_WORDS = or_(
    rule(normalized('дом')),
    rule(
        caseless('д'),
        DOT.optional()
    )
).interpretation(
    Building.type.const('дом')
)

DOM_VALUE = ADDR_VALUE.interpretation(
    Building.number
)

DOM = rule(
    DOM_WORDS,
    DOM_VALUE
).interpretation(
    Building
)

#########
#
#  Place
#
##########

COMPLEX_PLACENAME = morph_pipeline([
    'рабочее место',
    'раб.место'
])

PLACENAME = or_(
    rule(normalized('место')),
    rule(
        caseless('мес'),
        DOT.optional()
    ),
    rule(caseless('м'), DOT),
    rule(caseless('рм'), DOT.optional()),
    rule(
        caseless('р'),
        DOT.optional(),
        caseless('м'),
        DOT.optional()
    ),
    COMPLEX_PLACENAME
).interpretation(
    work_place.type.const('место'))

PLACE_NUMBER = or_(
    rule(LETTERS, DOT_INT, DOT_INT),
    rule(INT, DOT_INT, DOT_INT.optional()),
    rule(LETTERS_INT, DOT_INT.optional()),
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS)
).interpretation(
    work_place.number
)

COMPLEX_PLACE = rule(COMPLEX_PLACENAME, 
         eq('№').optional(),
         PLACE_NUMBER)

PLACE = or_(
    rule(PLACENAME,
         eq('№').optional(),
         PLACE_NUMBER),
    COMPLEX_PLACE
).interpretation(
    work_place
)

ONLY_NUMBER_PLACE = or_(
    rule(INT, DOT_INT, DOT_INT.optional()), # 10.10.10
    rule(LETTERS_INTS, DOT_INT.optional()), # B2.10.10 , B.10.10, B2.10
    rule(INT, DOT, LETTERS_INTS), # 10.B10.10, 10.B.10
    rule(LETTERS_INTS, DOT_INT, DOT_INT.optional()), # B2.10.10, B.10.10, B2.10.10.10, B.10.10.10
    rule(INT, DOT, LETTERS, DOT_INT, DOT_INT.optional()), # 10.B.10, 10.B.10.10
).interpretation(
    OnlyNumberPlace.number
).interpretation(
    OnlyNumberPlace
)

###########
#
#  KORPUS IMP
#
##########


KORPUS_WORDS = or_(
    rule(
        in_caseless({'корп', 'кор'}),
        DOT.optional()
    ),
    rule(normalized('корпус'))
).interpretation(
    Building.type.const('корпус')
)

KORPUS_VALUE = or_(rule(ADDR_VALUE).interpretation(
    Building.number),
                   rule(LETTERS).interpretation(
    Building.number)
)

KORPUS = rule(
        KORPUS_WORDS,
        KORPUS_VALUE
).interpretation(
    Building
)

###########
#
#  STROENIE IMP
#
##########


STROENIE_WORDS = or_(
    rule(
        caseless('стр'),
        DOT.optional()
    ),
    rule(normalized('строение'))
).interpretation(
    Building.type.const('строение')
)

STROENIE_VALUE = ADDR_VALUE.interpretation(
    Building.number
)

STROENIE = rule(
    STROENIE_WORDS,
    STROENIE_VALUE
).interpretation(
    Building
)

###########
#
#   OFIS CPY
#
#############


OFIS_WORDS = or_(
    rule(
        caseless('оф'),
        DOT.optional()
    ),
    rule(normalized('офис'))
).interpretation(
    Room.type.const('офис')
)

OFIS_VALUE = ADDR_VALUE.interpretation(
    Room.number
)

OFIS = rule(
    OFIS_WORDS,
    OFIS_VALUE
).interpretation(
    Room
)


###########
#
#   KVARTIRA CPY
#
#############


KVARTIRA_WORDS = or_(
    rule(
        caseless('кв'),
        DOT.optional()
    ),
    rule(normalized('квартира'))
).interpretation(
    Room.type.const('квартира')
)

KVARTIRA_VALUE = ADDR_VALUE.interpretation(
    Room.number
)

KVARTIRA = rule(
    KVARTIRA_WORDS,
    KVARTIRA_VALUE
).interpretation(
    Room
)

#########
#
#  work_place_part (final rule)
#
##########

WORK_PLACE_PART = or_(
    INDEX,
    
    GOROD,
    FED_OKRUG,
    RESPUBLIKA,
    KRAI,
    OBLAST,
    AUTO_OKRUG,
    
    DEREVNYA,
    SELO,
    POSELOK,
    
    STREET,
    PROSPEKT,
    PROEZD,
    PEREULOK,
    PLOSHAD,
    SHOSSE,
    NABEREG,
    BULVAR,
    
    DOM,
    KORPUS,
    STROENIE,
    OFIS,
    KVARTIRA,
    
    BC,
    ONLY_NAME_BC,
    FLOOR,
    PLACE,
    ONLY_NUMBER_PLACE,
    SECTOR,
    ROOM,
    CABINET,
    WINDOW,
    RAION,
    ISU,
    ONLY_NUMBER_ISU
).interpretation(
    AddrPart.value
).interpretation(
    AddrPart
)

parser = Parser(WORK_PLACE_PART)

###Функция yargy

In [135]:
def yargy_parser(text, parser=Parser(WORK_PLACE_PART)):
    region = ''
    town = ''
    district = ''
    settlement = ''
    street = ''
    building = ''
    index = ''
    place = ''
    isnone = ''
    deleted_addr_parts = text
    i = 0

    tmp_one_place_list = []
    building_list = []
    work_place_list = ['БЦ', 'окно', 'ряд', 'место', 'кабинет', 'этаж', 'Отделение', 'ВСП/ДО', 'сектор', 'Блок', 'комната']
    street_list = ['улица', 'проспект', 'проезд', 'переулок', 'площадь', 'шоссе', 'набережная', 'бульвар']

    for match in parser.findall(text):
        if not match.fact.value.type is None:
            deleted_addr_parts = deleted_addr_parts[0: match.span.start - i] + deleted_addr_parts[match.span.stop - i:]
            i += match.span.stop - match.span.start
        if match.fact.value.type in work_place_list:
            tmp_one_place_list.append(match.fact.value.type)
            tmp_one_place_list.append(match.fact.value.value)
            place = ' '.join(tmp_one_place_list)
        elif match.fact.value.type in ('федеральный округ', 'республика', 'край', 'область', 'автономный округ'):
            region = match.fact.value.type + ' ' + match.fact.value.name
        elif match.fact.value.type == 'район':
            district = match.fact.value.name + ' ' + match.fact.value.type
        elif match.fact.value.type == 'город':
            town = match.fact.value.value
        elif match.fact.value.type in ('село', 'деревня', 'посёлок'):
            settlement = match.fact.value.type + ' ' + match.fact.value.name
        elif match.fact.value.type in street_list:
            street = match.fact.value.type + ' ' + match.fact.value.name
        elif match.fact.value.type in ('дом', 'корпус', 'строение', 'офис', 'квартира'):
            building_list.append(match.fact.value.type)
            building_list.append(match.fact.value.number)
            building = " ".join(building_list)
        elif match.fact.value.type == 'индекс':
            index = match.fact.value.value
        elif match.fact.value.type is None:
            isnone = match.fact.value.value
        
    return region, town, district, settlement, street, building, index, place, isnone, deleted_addr_parts#remove_duplicate_punctuation(deleted_addr_parts)

# Запуск yargy

In [136]:
d = data.copy()
# d = data[data.index==85647].copy()#

d['region'], d['town'], d['district'], d['settlement'], d['street'], \
d['building'], d['index'], d['place'], d['none'], d['else'] = zip(*d['else'].apply(yargy_parser))

d['town'] = data['town'].add(d['town'], fill_value="")
d['region'] = data['region'].add(d['region'], fill_value="")
d['settlement'] = data['settlement'].add(d['settlement'], fill_value="")

In [ ]:
 #182399 убрать склеивания возможно добавить села-поселки из 3 слов ( ) 99101

In [118]:
data[data.index==85647]

,address,region,town,district,settlement,street,building,index,place,none,else
85647,"г Одинцово , обл Московская , р-н Одинцовский ул Молодежная , 25 6 . 9",Московская,Одинцово,,,,,,,,", , р-н Одинцовский ул Молодежная , 25 6 . 9"


In [138]:
d[50:75]

,address,region,town,district,settlement,street,building,index,place,none,else
118948,климовск Ленина 1,,,,Климовск,,,,,,Ленина 1
52999,Михайловка Красноармейская 25,,Михайловка,,,,,,,,Красноармейская 25
65718,"пгт . Сарс Советская , 52",,,,Сарс,,,,,,". Советская , 52"
165097,"г Москва Дорогомилово , пр-кт Кутузовский , 32к2 openspace 6 . 01",,Москва,,,проспект Кутузовский,,,место 6 . 01,,"Дорогомилово , , 32к2 openspace"
53650,"Смоленск ул Н-Неман , 23",,Смоленск,,,улица Н-Неман,,,,,", 23"
163306,Якутск Октябрьская 17 303,,Якутск,,,,,,,Октябрьская,Октябрьская 17 303
170980,"Москва Вавилова , 19 В03 . 156",,Москва,,,,,,место В03 . 156,,"Вавилова , 19"
159659,Ивдель проспект Комсомола д . 30 окно 3,,Ивдель,,,проспект Комсомола,дом 30,,окно 3,,
149208,"Сочи Войкова , 2 211",,Сочи,,,,,,,,"Войкова , 2 211"
125423,г . Санкт-Петербург Литейный пр . д . 35 офис,,Санкт-петербург,,,проспект Литейный,дом 35,,,,. офис


In [139]:
sj = d.copy()

#Левенштейн

In [140]:
def levenshtein_towns(s, x_town, x_settlement, towns):
    if not (x_town == "" and x_settlement == ""):
        return s, x_town
    end_list = ("ий", "ый", "ая", "яя", "ое", "ее", "ой", "ва", "на", "са", "ма")
    res = list()
    key = True
    res_town = ""
    for word in s.lower().split():
        if key and (word[-2:] not in end_list or word[-2:] == "ва" and fuzz.ratio(word, "москва") >= 90):
            for town in towns:
                if fuzz.ratio(word, town.lower()) >= 90:
                    print(s)
                    print(word + " ---> " + town)
                    key = False
                    res_town = town
            if key:
                res.append(word)
        else:
            res.append(word)
    return " ".join(res), res_town

In [ ]:
d = sj.copy()

In [141]:
d['else'], d['town'] = zip(*d.apply(lambda x: levenshtein_towns(x['else'], x['town'], x['settlement'], towns=towns), axis=1))

In [142]:
d[(d['town']=="")&(d['settlement']=="")][0:20]

,address,region,town,district,settlement,street,building,index,place,none,else
176009,ВСП8619/00886,,,,,,,,ВСП/ДО 8619/00886,,
36151,"Первомайский Пр Улитина , д . 13а",,,,,проспект Первомайский,дом 13а,,,,"улитина ,"
7049,"пр-т 40 лет Октября , д . 20",,,,,проспект 40 лет Октября,дом 20,,,,","


In [ ]:
def final_parsing(town, settlement, street, building, s):
    if town == "" and settlement == "":
        ts = addr_part_extractor(s, names=SPECIAL_TOWNS + SETTLEMENTS_TO_REMOVE + tuple(settlements_adjf))
        if ts != "":
            print(s)
            print(ts)
            print("--------------------------")

In [ ]:
zip(*d.apply(lambda x: final_parsing(x['town'], x['settlement'], x['street'], x['building'], x['else']), axis=1))

малиновский ленина , 17
Малиновский
--------------------------
советская 67 1
Советская
--------------------------
ленина
Ленина
--------------------------
ст . ленинградская . 94 2 этаж ,
Ленинградская
--------------------------
большая морская 7
Большая
--------------------------
нехаевский р-он , ст упорниковская комсомольская 53 1
Нехаевский
--------------------------
в . новгород мира 32 к1
Новгород
--------------------------
ванинский , рп комсомольская , 3
Комсомольская
--------------------------
ст . советская , 85
Советская
--------------------------
н . новгород академика сахарова
Новгород
--------------------------
калиновское глазкова 218
Калиновское
--------------------------
интернациональная , 10
Интернациональная
--------------------------
октябрьская 23 операционный отдел
Октябрьская
--------------------------
рп октябрьский ленина 36 операционная часть
Октябрьский
--------------------------
тверское осб 7982
Тверское
--------------------------
ст . кавказская м . горь

TypeError: ignored

In [ ]:
SPECIAL_TOWNS = ('октябрьский',)

SETTLEMENT_KEY_WORDS = ("село", "с", "сл", "деревня", "дер", "поселок", "посёлок", "п", "пос", "рп", "пгт", "ст")
SETTLEMENTS_TO_REMOVE = ("пушкина", "ленина", "новгород", "революции", "люксембург", \
                         "энтузиастов", "ермакова", "мира", "чапаев", "ряд", "улица", \
                         "кирова", "гагарина", "энгельса", "калинина", "саха", "фрунзе")
SETTLEMENTS_TO_ADD = ("Зеленоград", "Винсады", "Вольск-18")
COMPLEX_SETTLEMENTS_TO_REMOVE = ("большая дмитровка", "1 мая", "8 марта", "карла маркса",\
                    "красный путь", "розы люксембург", "степана разина", "красный октябрь", \
                    "льва толстого", "красная площадь", "красные зори", "большая покровская", \
                    "максима горького", "красная пресня", "золотая долина")

# Сохранение результатов

In [ ]:
#добавить ставропольский край город кызыл пос Унъюган

In [ ]:
data.to_csv('address_parsing.csv', index=True)  

#Ненужное

In [ ]:
# def key_setter(s):
#     special_list = ("", "большая_дмитровка", "1_мая", "8_марта", "карла_маркса",\
#                     "красный_путь", "розы_люксембург", "степана_разина", "красный_октябрь", \
#                     "льва_толстого", "красная_площадь", "красные_зори", "большая_покровская", \
#                     "максима_горького", "красная_пресня", "", "", "", "", "", "", "", "", "", "", "" )
#     norm_list = ("теплый_стан", "марьина_роща", "белый_яр", "нижняя_тавда", "газимурский_завод", \
#                  "красный_яр", "высокий_мыс", "крутая_горка", "красная_поляна", "", "", "", "" )
#     if s in special_list or s in norm_list:
#         return 0
#     else:
#         return 1
# data['key'] = data['settlement'].apply(key_setter)

# def extracted_locs_split(locations):
#     result = list()
#     for loc in locations:
#         for word in loc.split():
#             result.append(word)
#     return result

# def extract_similar_to_complex(locs, complex_locs):
#     result = list()
#     complex_locs = list(map(str.lower, complex_locs))
#     for loc in locs:
#         if not loc.lower() in complex_locs:
#             result.append(loc)
#     return result

# complex_locations_list = list(set(extracted_locs_split(complex_locations)))
# locations = extract_similar_to_complex(locations, complex_locations_list)

#complex_list = list(locations_dict.values())

# %%time
# no_towns['else'] = no_towns['else'].apply(locs_changer, locs_dict=settlements_dict)

# import re
# # data['else'] = data.apply(lambda x: delete_addr_part(x['address'], (x['town'],), TOWNS_KEY_WORDS), axis=1)
# def remove_duplicate_punctuation(s):
#     res = list()
#     for word in s.split():
#         if res:
#             if res[-1] != ',' and res[-1] != '.' or word.isalnum():
#                 word = re.sub(',+', ',', word)
#                 word = re.sub('\.+', '.', word)
#                 res.append(word)
#         elif word.isalnum():
#                 res.append(word)
#     return " ".join(res)


# str1 = ",,, hello , , darling ., tutu ..,"
# str1 = remove_duplicate_punctuation(str1)
# print(str1)

In [ ]:
print(fuzz.ratio('Екатеринубрг', 'Екатеринбург'))

92
96
92
92
88


In [52]:
from natasha.grammars.addr import GOROD, INDEX, FED_OKRUG, RESPUBLIKA, KRAI, AUTO_OKRUG, DEREVNYA, \
SELO, POSELOK

from yargy import Parser
from IPython.display import display
from yargy import (
    rule,
    or_, and_
)
from yargy.interpretation import fact
from yargy.predicates import (
    eq, lte, gte, gram, type, tag,
    length_eq,
    in_, in_caseless, dictionary,
    normalized, caseless,
    is_title
)
from yargy.pipelines import morph_pipeline
from yargy.tokenizer import QUOTES

INT = type('INT')
DOT = eq('.')
ADJF = gram('ADJF')
NOUN = gram('NOUN')
TITLE = is_title()
DASH = eq('-')
SLASH = eq('/')
QUOTE = in_(QUOTES)

ANUM = rule(
    INT,
    DASH.optional(),
    in_caseless({
        'я', 'й', 'е',
        'ое', 'ая', 'ий', 'ой'
    })
)

def value(key):
    @property
    def field(self):
        return getattr(self, key)
    return field

work_place = fact(
    'work_place',
    ['number', 'type']
)

AddrPart = fact(
    'AddrPart',
    ['value']
)

work_place_part = fact(
    'AddrPart',
    ['value']
)

OnlyNumberIsu = fact(
    'OnlyNumberIsu',
    ['number']
)

OnlyNumberPlace = fact(
    'OnlyNumberPlace',
    ['number']
)

OnlyNameBC = fact(
    'OnlyNameBC',
    ['name']
)

Region = fact(
    'Region',
    ['name', 'type']
)

Building = fact(
    'Building',
    ['number', 'type']
)

Room = fact(
    'Room',
    ['number', 'type']
)

Settlement = fact(
    'Settlement',
    ['name', 'type']
)

Street = fact(
    'Street',
    ['name', 'type']
)

class Region(Region):
    value = value('name')

class Settlement(Settlement):
    value = value('name')
    
class Street(Settlement):
    value = value('name')

class Building(Building):
    value = value('number')
    
class Room(Room):
    value = value('number')

class OnlyNumberIsu(OnlyNumberIsu):
    type = 'ВСП/ДО'
    value = value('number')
    
class OnlyNumberPlace(OnlyNumberPlace):
    type = 'место'
    value = value('number')   
    
class OnlyNameBC(OnlyNameBC):
    type = 'БЦ'
    value = value('name')    
    
class work_place(work_place):
    value = value('number')   
    
class work_place_part(AddrPart):
    @property
    def obj(self):
        from natasha import obj

        part = self.value
        return obj.AddrPart(part.value, part.type)

from yargy.relations import gender_relation
from yargy.relations.constructors import Relation

class new_gender_relation(Relation):
    label = 'gender_fem'

    def __call__(self, form, other):
        if form.grams.number.plural and other.grams.number.plural:
            return True
        
        fem_types = ("ул", "пл", "наб")
        male_types = ("пр", "пр-кт", "пр-т", "пр-зд", "пр-д", \
                    "пер", "б-р", "б", "бул", "бр")
        neut_types = ("ш",)
        (form_male, form_female, form_neutral,
         form_bi, form_general) = form.grams.gender
        (other_male, other_female, other_neutral,
         other_bi, other_general) = other.grams.gender
        if form.normalized in fem_types:
            form_female = True
        if other.normalized in fem_types:
            other_female = True
        if other.normalized in male_types:
            form_male = True
        if other.normalized in male_types:
            other_male = True
        if form.normalized in neut_types:
            form_neutral = True
        if other.normalized in neut_types:
            other_neutral = True
        return (
            (form_male and other_male)
            or (form_female and other_female)
            or (form_neutral and other_neutral)
            or (form_bi and (other_male or other_female))
            or (other_bi and (form_male or form_female))
            or form_general
            or other_general)
        

#########
#
#  Number
#
##########

LETTER = in_caseless(set('абвгдежзийклмнопрстуфхшщэюя'))
LETTER_LATIN = in_caseless(set('qwertyuiopasdfghjklzxcvbnm'))
LETTERS = or_(
    rule(LETTER),
    rule(LETTER_LATIN)
)
SEP = in_(r'/\-')

SIMPLE_NUMBER = rule(
    INT,
    LETTERS.optional()
)

NUMBER_CASES = or_(
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS)
)

NUMBER = rule(
    eq('№').optional(),
    NUMBER_CASES
)

#########
#
#  BC
#
##########

SIMPLE_BC = or_(
        NOUN,
        ADJF
)

COMPLEX_BC = morph_pipeline([
    'poklonka place',
    'президент плаза',
    'поклонка плейс',
    'северное сияние',
    'даниловская мануфактура',
    'корпус кнопа',
    'чайка плаза',
    'романов двор-2',
    'золотая долина',
    'Фаберже 8'
])

COMPLEX_BC_WORDS = morph_pipeline([
    'бизнес-центр',
    'бизнес центр',
    'бизнес-парк'
])

SIMPLE_BC_KNOWN = dictionary({
    'кнопа',
    'ртс',
    'Чайка',
    'сенатор'
})

BC_ABBR = in_caseless({
    'ДМ'
})

ONLY_NAME_BC = or_(
   COMPLEX_BC,
   rule(SIMPLE_BC_KNOWN)
).interpretation(
    OnlyNameBC.name
).interpretation(
    OnlyNameBC
)

BC_NAME = or_(
    rule(SIMPLE_BC_KNOWN),
    rule(BC_ABBR),
    rule(QUOTE, SIMPLE_BC_KNOWN, QUOTE),
    rule(QUOTE, COMPLEX_BC, QUOTE),
    rule(QUOTE, BC_ABBR, QUOTE),
    rule(QUOTE, SIMPLE_BC, QUOTE),
    rule(SIMPLE_BC),
    rule(COMPLEX_BC)
).interpretation(
    Settlement.name
)

BC_WORDS = or_(
        rule(caseless('бц'),
        DOT.optional()),
        COMPLEX_BC_WORDS
).interpretation(
    Settlement.type.const('БЦ')
)

BC = or_(
    rule(BC_WORDS, BC_NAME),
    rule(
        COMPLEX_BC_WORDS,
        BC_NAME
    )
).interpretation(
    Settlement
)

#########
#
#  Floor
#
##########

FLOORNAME = rule(normalized('этаж')
).interpretation(
    work_place.type.const('этаж'))

FLOOR_NUMBER = NUMBER.interpretation(
    work_place.number
)

FLOOR = rule(
    FLOORNAME,
    FLOOR_NUMBER
).interpretation(
    work_place
)

#########
#
#  Cabinet
#
##########


CABNAME = or_(
    rule(normalized('кабинет')),
    rule(
        caseless('каб'),
        DOT.optional()
    ),
    rule(
        caseless('к'),
        DOT
    )
).interpretation(
    work_place.type.const('кабинет')
)

CAB_NUMBER = NUMBER.interpretation(
    work_place.number
)

CABINET = rule(
    CABNAME,
    CAB_NUMBER
).interpretation(
    work_place
)

#########
#
#  Sector
#
##########

DOT_INT = rule(DOT, INT)

SECTORNAME = or_(
    caseless('сектор'),
    caseless('блок')
).interpretation(
    work_place.type.const('сектор'))

LETTERS_OPT_INT = or_(
    rule(LETTER, INT.optional()),
    rule(LETTER_LATIN, INT.optional())
)

LETTERS_INT = or_(
    rule(LETTER, INT),
    rule(LETTER_LATIN, INT)
)

LETTERS_INTS = rule(LETTERS_OPT_INT, DOT_INT)

QUOTE_LETTER_INT = rule(
    QUOTE, LETTERS_OPT_INT, QUOTE
)

SECTOR_NUMBER = or_(
    rule(QUOTE_LETTER_INT),
    rule(LETTERS_INT, DOT_INT.optional()),
    rule(LETTERS_OPT_INT),
    rule(LETTERS, DOT_INT),
    rule(INT, DOT_INT, DOT_INT.optional()),
    rule(LETTERS, DOT_INT),
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS)
).interpretation(
    work_place.number
)

SECTOR = rule(
    SECTORNAME,
    eq('№').optional(),SECTOR_NUMBER
).interpretation(
    work_place
)

# #########
# #
# #  Place
# #
# ##########

# COMPLEX_PLACENAME = morph_pipeline([
#     'рабочее место',
#     'раб.место'
# ])

# PLACENAME = or_(
#     rule(normalized('место')),
#     rule(
#         caseless('мес'),
#         DOT.optional()
#     ),
#     rule(caseless('м'), DOT),
#     rule(caseless('рм'), DOT.optional()),
#     rule(
#         caseless('р'),
#         DOT.optional(),
#         caseless('м'),
#         DOT.optional()
#     ),
#     COMPLEX_PLACENAME
# ).interpretation(
#     work_place.type.const('место'))

# PLACE_NUMBER = or_(
#     rule(LETTERS, DOT_INT, DOT_INT),
#     rule(INT, DOT_INT, DOT_INT.optional()),
#     rule(LETTERS_INT, DOT_INT.optional()),
#     rule(SIMPLE_NUMBER),
#     rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
#     rule(SIMPLE_NUMBER, SEP, LETTERS)
# ).interpretation(
#     work_place.number
# )

# COMPLEX_PLACE = rule(COMPLEX_PLACENAME, 
#          eq('№').optional(),
#          PLACE_NUMBER)

# PLACE = or_(
#     rule(PLACENAME,
#          eq('№').optional(),
#          PLACE_NUMBER),
#     COMPLEX_PLACE
# ).interpretation(
#     work_place
# )

# ONLY_NUMBER_PLACE = or_(
#     rule(INT, DOT_INT, DOT_INT.optional()), # 10.10.10
#     rule(LETTERS_INTS, DOT_INT.optional()), # B2.10.10 , B.10.10, B2.10
#     rule(INT, DOT, LETTERS_INTS), # 10.B10.10, 10.B.10
#     rule(LETTERS_INTS, DOT_INT, DOT_INT.optional()), # B2.10.10, B.10.10, B2.10.10.10, B.10.10.10
#     rule(INT, DOT, LETTERS, DOT_INT, DOT_INT.optional()), # 10.B.10, 10.B.10.10
# ).interpretation(
#     OnlyNumberPlace.number
# ).interpretation(
#     OnlyNumberPlace
# )

#########
#
#  Room
#
##########

ROOMNAME = or_(
    rule(caseless('комната')),
    rule(caseless('ком'),
         DOT.optional()),
    rule(caseless('комн'),
         DOT.optional())
).interpretation(
    work_place.type.const('комната'))

ROOM_NUMBER = or_(
    rule(LETTERS_INT),
    rule(INT, DOT_INT, DOT_INT.optional()),
    rule(SIMPLE_NUMBER)
).interpretation(
    work_place.number
)

ROOM = rule(
    ROOMNAME,
    eq('№').optional(),ROOM_NUMBER
).interpretation(
    work_place
)

#########
#
#  Window
#
##########

WINDOWNAME = rule(normalized('окно')
).interpretation(
    work_place.type.const('окно'))

WINDOW_NUMBER = NUMBER.interpretation(
    work_place.number
)

WINDOW = rule(
    WINDOWNAME,
    WINDOW_NUMBER
).interpretation(
    work_place
)

#########
#
#  ВСП
#
##########

INT_FOUR = and_(INT,
                length_eq(4))

ONLY_NUMBER_ISU = rule(
    INT_FOUR,
    SLASH,
    INT
).interpretation(
    OnlyNumberIsu.number
).interpretation(
    OnlyNumberIsu
)


COMPLEX_ISUNAME = morph_pipeline([
    'дополнительный офис',
    'доп.офис'
])

ISUNAME = or_(
    rule(caseless('ВСП')),
    rule(caseless('ДО')),
    COMPLEX_ISUNAME
).interpretation(
    work_place.type.const('ВСП/ДО'))

ISU_NUMBER = or_(
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS),
).interpretation(
    work_place.number
)

COMPLEX_ISU = rule(COMPLEX_ISUNAME, 
         eq('№').optional(),
         ISU_NUMBER)

ISU = or_(
    rule(ISUNAME,
         eq('№').optional(),
         ISU_NUMBER),
    rule(COMPLEX_ISU)
).interpretation(
    work_place
)

############
#
#    OBLAST IMP
#
############


OBLAST_WORDS = or_(
    rule(normalized('область')),
    rule(
        caseless('обл'),
        DOT.optional()
    )
).interpretation(
    Region.type.const('область')
)

OBLAST_NAME = dictionary({
    'амурский',
    'архангельский',
    'астраханский',
    'белгородский',
    'брянский',
    'владимирский',
    'волгоградский',
    'вологодский',
    'воронежский',
    'горьковский',
    'ивановский',
    'ивановский',
    'иркутский',
    'калининградский',
    'калужский',
    'камчатский',
    'кемеровский',
    'кировский',
    'костромской',
    'курганский',
    'курский',
    'ленинградский',
    'липецкий',
    'магаданский',
    'московский',
    'мурманский',
    'нижегородский',
    'новгородский',
    'новосибирский',
    'омский',
    'оренбургский',
    'орловский',
    'пензенский',
    'пермский',
    'псковский',
    'ростовский',
    'рязанский',
    'самарский',
    'саратовский',
    'сахалинский',
    'свердловский',
    'смоленский',
    'тамбовский',
    'тверской',
    'томский',
    'тульский',
    'тюменский',
    'ульяновский',
    'челябинский',
    'читинский',
    'ярославский',
}).interpretation(
    Region.name
)

OBLAST = or_(
    rule(OBLAST_NAME, OBLAST_WORDS),
    rule(OBLAST_WORDS, OBLAST_NAME)
).interpretation(
    Region
)

##########
#
#  RAION IMP
#
###########


RAION_WORDS = or_(
    rule(caseless('р'), '-', in_caseless({'он', 'н'})),
    rule(caseless('мрн')),
    rule(caseless('мкр')),
    rule(normalized('район')),
    rule(normalized('микрорайон'))
).interpretation(
    Region.type.const('район')
)

RAION_SIMPLE_NAME = and_(
    ADJF,
    TITLE
)

RAION_MODIFIERS = rule(
    in_caseless({
        'усть',
        'северо',
        'александрово',
        'гаврилово',
    }),
    DASH.optional(),
    TITLE
)

RAION_COMPLEX_NAME = rule(
    RAION_MODIFIERS,
    RAION_SIMPLE_NAME
)

RAION_NAME = or_(
    rule(RAION_SIMPLE_NAME),
    RAION_COMPLEX_NAME
).interpretation(
    Region.name
)

RAION = rule(
    RAION_NAME,RAION_WORDS
    #rule(RAION_WORDS,RAION_NAME)
).interpretation(
    Region
)

##############
#
#   ADDR PERSON CPY
#
############


ABBR = and_(
    length_eq(1),
    is_title()
)

PART = and_(
    TITLE,
    or_(
        gram('Name'),
        gram('Surn')
    )
)

MAYBE_FIO = or_(
    rule(TITLE, PART),
    rule(PART, TITLE),
    rule(ABBR, '.', TITLE),
    rule(ABBR, '.', ABBR, '.', TITLE),
    rule(TITLE, ABBR, '.', ABBR, '.')
)

POSITION_WORDS_ = or_(
    rule(
        dictionary({
            'мичман',
            'геолог',
            'подводник',
            'краевед',
            'снайпер',
            'штурман',
            'бригадир',
            'учитель',
            'политрук',
            'военком',
            'ветеран',
            'историк',
            'пулемётчик',
            'авиаконструктор',
            'адмирал',
            'академик',
            'актер',
            'актриса',
            'архитектор',
            'атаман',
            'врач',
            'воевода',
            'генерал',
            'губернатор',
            'хирург',
            'декабрист',
            'разведчик',
            'граф',
            'десантник',
            'конструктор',
            'скульптор',
            'писатель',
            'поэт',
            'капитан',
            'князь',
            'комиссар',
            'композитор',
            'космонавт',
            'купец',
            'лейтенант',
            'лётчик',
            'майор',
            'маршал',
            'матрос',
            'подполковник',
            'полковник',
            'профессор',
            'сержант',
            'старшина',
            'танкист',
            'художник',
            'герой',
            'княгиня',
            'строитель',
            'дружинник',
            'диктор',
            'прапорщик',
            'артиллерист',
            'графиня',
            'большевик',
            'патриарх',
            'сварщик',
            'офицер',
            'рыбак',
            'брат',
        })
    ),
    rule(normalized('генерал'), normalized('армия')),
    rule(normalized('герой'), normalized('россия')),
    rule(
        normalized('герой'),
        normalized('российский'), normalized('федерация')),
    rule(
        normalized('герой'),
        normalized('советский'), normalized('союз')
    ),
)

ABBR_POSITION_WORDS = rule(
    in_caseless({
        'адм',
        'ак',
        'акад',
    }),
    DOT.optional()
)

POSITION_WORDS = or_(
    POSITION_WORDS_,
    ABBR_POSITION_WORDS
)

MAYBE_PERSON = or_(
    MAYBE_FIO,
    rule(POSITION_WORDS, MAYBE_FIO),
    rule(POSITION_WORDS, TITLE)
)

###########
#
#   IMENI CPY
#
##########


IMENI_WORDS = or_(
    rule(
        caseless('им'),
        DOT.optional()
    ),
    rule(caseless('имени'))
)

IMENI = or_(
    rule(
        IMENI_WORDS.optional(),
        MAYBE_PERSON
    ),
    rule(
        IMENI_WORDS,
        TITLE
    )
)

##########
#
#   LET CPY
#
##########


LET_WORDS = or_(
    rule(caseless('лет')),
    rule(
        DASH.optional(),
        caseless('летия')
    )
)

LET_NAME = in_caseless({
    'влксм',
    'ссср',
    'алтая',
    'башкирии',
    'бурятии',
    'дагестана',
    'калмыкии',
    'колхоза',
    'комсомола',
    'космонавтики',
    'москвы',
    'октября',
    'пионерии',
    'победы',
    'приморья',
    'района',
    'совхоза',
    'совхозу',
    'татарстана',
    'тувы',
    'удмуртии',
    'улуса',
    'хакасии',
    'целины',
    'чувашии',
    'якутии',
})

LET = rule(
    INT,
    LET_WORDS,
    LET_NAME
)

##########
#
#    ADDR DATE CPY
#
#############


MONTH_WORDS = dictionary({
    'январь',
    'февраль',
    'март',
    'апрель',
    'май',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь',
})

DAY = and_(
    INT,
    gte(1),
    lte(31)
)

YEAR = and_(
    INT,
    gte(1),
    lte(2100)
)

YEAR_WORDS = normalized('год')

DATE = or_(
    rule(DAY, MONTH_WORDS),
    rule(YEAR, YEAR_WORDS)
)

#########
#
#   MODIFIER CPY
#
############


MODIFIER_WORDS_ = rule(
    dictionary({
        'большой',
        'малый',
        'средний',

        'верхний',
        'центральный',
        'нижний',
        'северный',
        'дальний',

        'первый',
        'второй',

        'старый',
        'новый',

        'красный',
        'лесной',
        'тихий',
    }),
    DASH.optional()
)

ABBR_MODIFIER_WORDS = rule(
    in_caseless({
        'б', 'м', 'н'
    }),
    DOT.optional()
)

SHORT_MODIFIER_WORDS = rule(
    in_caseless({
        'больше',
        'мало',
        'средне',

        'верх',
        'верхне',
        'центрально',
        'нижне',
        'северо',
        'дальне',
        'восточно',
        'западно',

        'перво',
        'второ',

        'старо',
        'ново',

        'красно',
        'тихо',
        'горно',
    }),
    DASH.optional()
)

MODIFIER_WORDS = or_(
    MODIFIER_WORDS_,
    ABBR_MODIFIER_WORDS,
    SHORT_MODIFIER_WORDS,
)


##########
#
#   ADDR NAME IMP
#
##########


ROD = gram('gent')

# SIMPLE = or_(
#         ADJF,  # Школьная
#         and_(NOUN, ROD),  # Ленина, Победы
#     )
# )

SIMPLE_NOUN = and_(NOUN, ROD)

COMPLEX = or_(
    rule(
        and_(ADJF),
        NOUN
    ),
    rule(
        TITLE,
        DASH.optional(),
        TITLE
    ),
)

EXCEPTION = dictionary({
    'арбат',
    'варварка',
    'мельникайте',
    'каховка',
    'зорге'
})

MAYBE_NAME = or_(
    #rule(SIMPLE),
    rule(SIMPLE_NOUN),
    COMPLEX,
    rule(EXCEPTION)
)

NAME = or_(
    MAYBE_NAME,
    LET,
    DATE,
    IMENI
)

NAME = rule(
    MODIFIER_WORDS.optional(),
    NAME
)

ADDR_CRF = tag('I').repeatable()

NAME = or_(
    NAME,
    ANUM,
    rule(NAME, ANUM),
    rule(ANUM, NAME),
    rule(INT, DASH.optional(), NAME),
    rule(NAME, DASH, INT),
    ADDR_CRF
)

ADDR_NAME = NAME


########
#
#    STREET IMP
#
#########

gender = new_gender_relation()

STREET_WORDS = or_(
    rule(normalized('улица')),
    rule(
        caseless('ул'),
        DOT.optional()
    )
).interpretation(
    Street.type.const('улица')
).match(gender)


STREET_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

STREET = or_(
    rule(STREET_WORDS, STREET_NAME),
    rule(STREET_NAME, STREET_WORDS)
).interpretation(
    Street
)

##########
#
#    PROSPEKT IMP
#
##########


PROSPEKT_WORDS = or_(
    rule(
        in_caseless({'пр', 'просп'}),
        DOT.optional()
    ),
    rule(
        caseless('пр'),
        '-',
        in_caseless({'кт', 'т'}),
        DOT.optional()
    ),
    rule(normalized('проспект'))
).interpretation(
    Street.type.const('проспект')
).match(gender)

PROSPEKT_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PROSPEKT = or_(
    rule(PROSPEKT_WORDS, PROSPEKT_NAME),
    rule(PROSPEKT_NAME, PROSPEKT_WORDS)
).interpretation(
    Street
)


############
#
#    PROEZD IMP
#
#############


PROEZD_WORDS = or_(
    rule(caseless('пр'), DOT.optional()),
    rule(
        caseless('пр'),
        '-',
        in_caseless({'зд', 'д'}),
        DOT.optional()
    ),
    rule(normalized('проезд'))
).interpretation(
    Street.type.const('проезд')
).match(gender)

PROEZD_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PROEZD = or_(
    rule(PROEZD_WORDS, PROEZD_NAME),
    rule(PROEZD_NAME, PROEZD_WORDS)
).interpretation(
    Street
)


###########
#
#   PEREULOK IMP
#
##############


PEREULOK_WORDS = or_(
    rule(
        caseless('п'),
        DOT
    ),
    rule(
        caseless('пер'),
        DOT.optional()
    ),
    rule(normalized('переулок'))
).interpretation(
    Street.type.const('переулок')
).match(gender)

PEREULOK_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PEREULOK = or_(
    rule(PEREULOK_WORDS, PEREULOK_NAME),
    rule(PEREULOK_NAME, PEREULOK_WORDS)
).interpretation(
    Street
)


########
#
#  PLOSHAD IMP
#
##########


PLOSHAD_WORDS = or_(
    rule(
        caseless('пл'),
        DOT.optional()
    ),
    rule(normalized('площадь'))
).interpretation(
    Street.type.const('площадь')
).match(gender)

PLOSHAD_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

PLOSHAD = or_(
    rule(PLOSHAD_WORDS, PLOSHAD_NAME),
    rule(PLOSHAD_NAME, PLOSHAD_WORDS)
).interpretation(
    Street
)

########
#
#  ADDR VALUE IMP
#
##########

LETTER = or_(
    rule(LETTER),
    rule(QUOTE, LETTER, QUOTE)
)

VALUE = rule(
    INT,
    LETTER.optional()
)

SEP = in_(r'/\-')

VALUE = or_(
    rule(VALUE),
    rule(VALUE, SEP, VALUE),
    rule(VALUE, SEP, LETTER),
)

ADDR_VALUE = rule(
    eq('№').optional(),
    VALUE
)

############
#
#   SHOSSE IMP
#
###########

SHOSSE_WORDS = or_(
    rule(
        caseless('ш'),
        DOT.optional()
    ),
    rule(normalized('шоссе'))
).interpretation(
    Street.type.const('шоссе')
).match(gender)

SHOSSE_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

SHOSSE = or_(
    rule(SHOSSE_NAME, SHOSSE_WORDS),
    rule(SHOSSE_WORDS, SHOSSE_NAME)
).interpretation(
    Street
)

########
#
#  NABEREG IMP
#
##########


NABEREG_WORDS = or_(
    rule(
        caseless('наб'),
        DOT.optional()
    ),
    rule(normalized('набережная'))
).interpretation(
    Street.type.const('набережная')
).match(gender)

NABEREG_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

NABEREG = or_(
    rule(NABEREG_WORDS, NABEREG_NAME),
    rule(NABEREG_NAME, NABEREG_WORDS)
).interpretation(
    Street
)


########
#
#  BULVAR IMP
#
##########


BULVAR_WORDS = or_(
    rule(
        caseless('б'),
        '-',
        caseless('р'),
        DOT.optional()
    ),
    rule(
        caseless('б'),
        DOT
    ),
    rule(
        caseless('бул'),
        DOT.optional()
    ),
    rule(normalized('бульвар'))
).interpretation(
    Street.type.const('бульвар')
).match(gender)

BULVAR_NAME = or_(rule(ADJF).interpretation(
    Street.name).match(gender),
                  rule(ADDR_NAME).interpretation(
    Street.name)
    )

BULVAR = or_(
    rule(BULVAR_WORDS, BULVAR_NAME),
    rule(BULVAR_NAME, BULVAR_WORDS)
).interpretation(
    Street
)

############
#
#    DOM IMP
#
#############


DOM_WORDS = or_(
    rule(normalized('дом')),
    rule(
        caseless('д'),
        DOT.optional()
    )
).interpretation(
    Building.type.const('дом')
)

DOM_VALUE = ADDR_VALUE.interpretation(
    Building.number
)

DOM = rule(
    DOM_WORDS,
    DOM_VALUE
).interpretation(
    Building
)

#########
#
#  Place
#
##########

COMPLEX_PLACENAME = morph_pipeline([
    'рабочее место',
    'раб.место'
])

PLACENAME = or_(
    rule(normalized('место')),
    rule(
        caseless('мес'),
        DOT.optional()
    ),
    rule(caseless('м'), DOT),
    rule(caseless('рм'), DOT.optional()),
    rule(
        caseless('р'),
        DOT.optional(),
        caseless('м'),
        DOT.optional()
    ),
    COMPLEX_PLACENAME
).interpretation(
    work_place.type.const('место'))

PLACE_NUMBER = or_(
    rule(LETTERS, DOT_INT, DOT_INT),
    rule(INT, DOT_INT, DOT_INT.optional()),
    rule(LETTERS_INT, DOT_INT.optional()),
    rule(SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, SIMPLE_NUMBER),
    rule(SIMPLE_NUMBER, SEP, LETTERS)
).interpretation(
    work_place.number
)

COMPLEX_PLACE = rule(COMPLEX_PLACENAME, 
         eq('№').optional(),
         PLACE_NUMBER)

PLACE = or_(
    rule(PLACENAME,
         eq('№').optional(),
         PLACE_NUMBER),
    COMPLEX_PLACE
).interpretation(
    work_place
)

ONLY_NUMBER_PLACE = or_(
    rule(INT, DOT_INT, DOT_INT.optional()), # 10.10.10
    rule(LETTERS_INTS, DOT_INT.optional()), # B2.10.10 , B.10.10, B2.10
    rule(INT, DOT, LETTERS_INTS), # 10.B10.10, 10.B.10
    rule(LETTERS_INTS, DOT_INT, DOT_INT.optional()), # B2.10.10, B.10.10, B2.10.10.10, B.10.10.10
    rule(INT, DOT, LETTERS, DOT_INT, DOT_INT.optional()), # 10.B.10, 10.B.10.10
).interpretation(
    OnlyNumberPlace.number
).interpretation(
    OnlyNumberPlace
)

###########
#
#  KORPUS IMP
#
##########


KORPUS_WORDS = or_(
    rule(
        in_caseless({'корп', 'кор'}),
        DOT.optional()
    ),
    rule(normalized('корпус'))
).interpretation(
    Building.type.const('корпус')
)

KORPUS_VALUE = ADDR_VALUE.interpretation(
    Building.number
)

KORPUS = rule(
        KORPUS_WORDS,
        KORPUS_VALUE
).interpretation(
    Building
)

###########
#
#  STROENIE IMP
#
##########


STROENIE_WORDS = or_(
    rule(
        caseless('стр'),
        DOT.optional()
    ),
    rule(normalized('строение'))
).interpretation(
    Building.type.const('строение')
)

STROENIE_VALUE = ADDR_VALUE.interpretation(
    Building.number
)

STROENIE = rule(
    STROENIE_WORDS,
    STROENIE_VALUE
).interpretation(
    Building
)

###########
#
#   OFIS CPY
#
#############


OFIS_WORDS = or_(
    rule(
        caseless('оф'),
        DOT.optional()
    ),
    rule(normalized('офис'))
).interpretation(
    Room.type.const('офис')
)

OFIS_VALUE = ADDR_VALUE.interpretation(
    Room.number
)

OFIS = rule(
    OFIS_WORDS,
    OFIS_VALUE
).interpretation(
    Room
)


###########
#
#   KVARTIRA CPY
#
#############


KVARTIRA_WORDS = or_(
    rule(
        caseless('кв'),
        DOT.optional()
    ),
    rule(normalized('квартира'))
).interpretation(
    Room.type.const('квартира')
)

KVARTIRA_VALUE = ADDR_VALUE.interpretation(
    Room.number
)

KVARTIRA = rule(
    KVARTIRA_WORDS,
    KVARTIRA_VALUE
).interpretation(
    Room
)

#########
#
#  work_place_part (final rule)
#
##########

WORK_PLACE_PART = or_(
    INDEX,
    
    GOROD,
    FED_OKRUG,
    RESPUBLIKA,
    KRAI,
    OBLAST,
    AUTO_OKRUG,
    
    DEREVNYA,
    SELO,
    POSELOK,
    
    STREET,
    PROSPEKT,
    PROEZD,
    PEREULOK,
    PLOSHAD,
    SHOSSE,
    NABEREG,
    BULVAR,
    
    DOM,
    KORPUS,
    STROENIE,
    OFIS,
    KVARTIRA,
    
    BC,
    ONLY_NAME_BC,
    FLOOR,
    PLACE,
    ONLY_NUMBER_PLACE,
    SECTOR,
    ROOM,
    CABINET,
    WINDOW,
    RAION,
    ISU,
    ONLY_NUMBER_ISU
).interpretation(
    AddrPart.value
).interpretation(
    AddrPart
)

parser = Parser(WORK_PLACE_PART)